In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as Fun
from tqdm import tqdm

devic = torch.device("cuda")

In [ ]:
torch.cuda.empty_cache()

# Question 1

## Self Attention Module

In [23]:
#------------------------------------------------------first question
class attention:
    def __init__(self,c,device):
        self.device = torch.device(device)
        self.Wq = torch.randn(c,c//8,device=device)
        self.Wk = torch.randn(c,c//8,device=device)
        self.Wv = torch.randn(c,c,device=device)
        self.Wq.requires_grad_()
        self.Wk.requires_grad_()
        self.Wv.requires_grad_()
    def forward(self,x,device):
        self.device = torch.device(device)
#         x1 = (x.view(x.size()[0],x.size()[2]*x.size()[1])).t().to(self.device)
#         print(x.size())
        Q = torch.matmul(x,self.Wq).to(self.device)
        K = torch.matmul(x,self.Wk).to(self.device)
#         sqd = torch.tensor((x.size()[1]+0.001)**0.5) /sqd.to(self.device)
        E = torch.matmul(Q,K.t())
        A = torch.softmax(E, dim=1).to(self.device)
        V = torch.matmul(x,self.Wv).to(self.device)
        Y = torch.matmul(A,V).to(self.device)
        return Y

### Self attention on CNN

In [29]:
class CNN_Transformer_(nn.Module):
    def __init__(self,device):
        
        super(CNN_Transformer_, self).__init__()
        
        self.device = torch.device(device)
        self.ReLU = nn.ReLU()
        
        self.conv1 = nn.Conv2d(3, 32, 5,device=device)
        self.atten1 = attention(32,self.device)
        self.norm1 = nn.LayerNorm(normalized_shape = 32, device = device)
        
        self.conv2 = nn.Conv2d(32, 64, 5,device=device)
        self.atten2 = attention(64,self.device)
        self.norm2 = nn.LayerNorm(normalized_shape = 64, device = device)
        self.pool = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(64, 128, 5,device=device)
        self.atten3 = attention(128,self.device)
        self.norm3 = nn.LayerNorm(normalized_shape = 128, device = device)
        
        self.conv4 = nn.Conv2d(128, 256, 5,device=device)
        #GAP here
        self.mlp1 = nn.Linear(256,125,device=device)
        self.mlp2 = nn.Linear(125,63,device=device)
        self.mlp3 = nn.Linear(63,10,device=device)
        
    def forward(self,x:torch.Tensor,device:str):
    
        self.device = torch.device(device)
        
        out1 = self.conv1(x).to(self.device)
        o = out1.view(  out1.size()[0]  ,  out1.size()[1]*out1.size()[2]  ).t()
        out2 = self.atten1.forward(o,self.device).to(self.device)
        out2 = torch.reshape(out2.t(),(out1.size()[1],out1.size()[2],out1.size()[0]))
        out2 = self.norm1(out2)
        out2 = torch.reshape(out2,(out1.size()[0],out1.size()[1],out1.size()[2]))
        out2 = torch.add(out1,out2)
        
        
        out3 = self.conv2(out2).to(self.device)
        o = out3.view(out3.size()[0],out3.size()[1]*out3.size()[2]).t()
        out4 = self.atten2.forward(o,self.device).to(self.device)
        out4 = torch.reshape(out4.t(),(out3.size()[1],out3.size()[2],out3.size()[0]))
        out4 = self.norm2(out4)
        out4 = torch.reshape(out4,(out3.size()[0],out3.size()[1],out3.size()[2]))
        out4 = torch.add(out3,out4)#residual network
        
        
        out4 = self.pool(out4)
        
        out5 = self.conv3(out4).to(self.device)
        o = out5.view(out5.size()[0],out5.size()[1]*out5.size()[2]).t()
        out6 = self.atten3.forward(o,self.device).to(self.device)
        out6 = torch.reshape(out6.t(),(out5.size()[1],out5.size()[2],out5.size()[0]))
        out6 = self.norm3(out6)
        out6 = torch.reshape(out6,(out5.size()[0],out5.size()[1],out5.size()[2]))
        out6 = torch.add(out5,out6)#residual network
        
        out7 = self.conv4(out6).to(self.device)
        out8 = (torch.einsum('ijk->i',out7))/(torch.tensor(out7.size()[1]*out7.size()[2])).to(self.device)
        out9 = self.ReLU(self.mlp1(out8))
        out10 = self.ReLU(self.mlp2(out9))
        out11 = Fun.softmax(self.mlp3(out10),dim=0).to(self.device)
        return out11

# Importing Data

In [30]:
import torchvision
from torchvision import transforms
#import CIFAR10

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

Train= torchvision.datasets .CIFAR10(root='./data',train = True,download=True,transform=transform)
Test= torchvision.datasets .CIFAR10(root='./data',train = False,download=True,transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [31]:
train_loader = torch.utils.data.DataLoader(dataset = Train,batch_size=1,shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = Test, batch_size=1,shuffle = False)

using cross entropy loss i.e;

$\Rightarrow J=-\sum_{i=0}^{9}(p_i \ast log(p_i))$ for each training point

$\Rightarrow J = -p_i*log(p_i)$ ,where $i$ is the index at which ground truth occurs


In [40]:

model = CNN_Transformer_('cuda')
model.to(devic)

CNN_Transformer_(
  (ReLU): ReLU()
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
  (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (norm3): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (conv4): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1))
  (mlp1): Linear(in_features=256, out_features=125, bias=True)
  (mlp2): Linear(in_features=125, out_features=63, bias=True)
  (mlp3): Linear(in_features=63, out_features=10, bias=True)
)

In [41]:
epochs = 3
lr = 0.01
batch_size = 32
for i in tqdm(range(epochs)):
    for j , (images, label) in enumerate(train_loader):

        images = images[0].to(devic)
        label = label.to(devic)
        p = model.forward(images,'cuda')
        lp = -torch.log(p).to(devic)
        
        Loss = lp[label].to(devic)
        
        if j%10000 == 0:
            print(Loss)
        
        Loss.backward()
        
    
        if((j+1)%batch_size == 0 or (j+1)==50000):
            with torch.no_grad():
                model.mlp3.weight.data -= lr*(model.mlp3.weight.grad)/batch_size
                model.mlp3.bias.data -= lr*(model.mlp3.bias.grad)/batch_size

                model.mlp2.weight.data -= lr*(model.mlp2.weight.grad)/batch_size
                model.mlp2.bias.data -= lr*(model.mlp2.bias.grad)/batch_size

                model.mlp1.weight.data -= lr*(model.mlp1.weight.grad)/batch_size
                model.mlp1.bias.data -= lr*(model.mlp1.bias.grad)/batch_size

                model.conv4.weight.data -= lr*(model.conv4.weight.grad)/batch_size
                model.conv4.bias.data -= lr*(model.conv4.bias.grad)/batch_size

                model.atten3.Wq.data -= lr*(model.atten3.Wq.grad)/batch_size
                model.atten3.Wk.data -= lr*(model.atten3.Wk.grad)/batch_size
                model.atten3.Wv.data -= lr*(model.atten3.Wv.grad)/batch_size

                model.conv3.weight.data -= lr*(model.conv3.weight.grad)/batch_size
                model.conv3.bias.data -= lr*(model.conv3.bias.grad)/batch_size

                model.atten2.Wq.data -= lr*(model.atten2.Wq.grad)/batch_size
                model.atten2.Wk.data -= lr*(model.atten2.Wk.grad)/batch_size
                model.atten2.Wv.data -= lr*(model.atten2.Wv.grad)/batch_size

                model.conv2.weight.data -= lr*(model.conv2.weight.grad)/batch_size
                model.conv2.bias.data -= lr*(model.conv2.bias.grad)/batch_size

                model.atten1.Wq.data -= lr*(model.atten1.Wq.grad)/batch_size
                model.atten1.Wk.data -= lr*(model.atten1.Wk.grad)/batch_size
                model.atten1.Wv.data -= lr*(model.atten1.Wv.grad/batch_size)

                model.conv1.weight.data -= lr*(model.conv1.weight.grad)/batch_size
                model.conv1.bias.data -= lr*(model.conv1.bias.grad)/batch_size



            model.mlp3.weight.grad.zero_()
            model.mlp3.bias.grad.zero_()

            model.mlp2.weight.grad.zero_()
            model.mlp2.bias.grad.zero_()

            model.mlp1.weight.grad.zero_()
            model.mlp1.bias.grad.zero_()

            model.conv4.weight.grad.zero_()
            model.conv4.bias.grad.zero_()

            model.atten3.Wq.grad.zero_()
            model.atten3.Wk.grad.zero_()
            model.atten3.Wv.grad.zero_()

            model.conv3.weight.grad.zero_()
            model.conv3.bias.grad.zero_()

            model.atten2.Wq.grad.zero_()
            model.atten2.Wk.grad.zero_()
            model.atten2.Wv.grad.zero_()


            model.conv2.weight.grad.zero_()
            model.conv2.bias.grad.zero_()

            model.atten1.Wq.grad.zero_()
            model.atten1.Wk.grad.zero_()
            model.atten1.Wv.grad.zero_()

            model.conv1.weight.grad.zero_()
            model.conv1.bias.grad.zero_()


  0%|          | 0/3 [00:00<?, ?it/s]

tensor([2.3518], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([1.7155], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([0.4370], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([2.0364], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([2.3115], device='cuda:0', grad_fn=<IndexBackward0>)


 33%|███▎      | 1/3 [04:22<08:45, 262.70s/it]

tensor([1.8351], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([0.7792], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([2.6258], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([1.6567], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([0.6100], device='cuda:0', grad_fn=<IndexBackward0>)


 67%|██████▋   | 2/3 [08:43<04:21, 261.59s/it]

tensor([1.8040], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([1.5332], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([1.9919], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([5.0020], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([2.3589], device='cuda:0', grad_fn=<IndexBackward0>)


100%|██████████| 3/3 [13:04<00:00, 261.52s/it]


In [42]:

correct = torch.tensor(0)
loss = torch.tensor(0.00000001)
for i , (images, label) in enumerate(test_loader):

    images = images[0].to(devic)
    label = label.to(devic)
    if (i%1000==0) :
        print(i//1000)
    p = model.forward(images,'cuda')
#     lp = -torch.log(p).to('cpu')
#     print(loss.device)
#     loss  -= torch.max(lp)
    
    if(int(torch.argmax(p))==label):
        correct = correct+1
        
#     del out
#     torch.cuda.empty_cache()
    
test_accuracy = correct/10000

0
1
2
3
4
5
6
7
8
9


In [43]:
test_accuracy*100

tensor(39.7800)

# Question 2

In [16]:
class multi_attention:
    def __init__(self,d:int,h:int,device:str):
        self.device = torch.device(device)
        self.Wq = torch.randn(h,d//h,d//h,device=device)
        self.Wk = torch.randn(h,d//h,d//h,device=device)
        self.Wv = torch.randn(h,d//h,d//h,device=device)
        self.Wq.requires_grad_()
        self.Wk.requires_grad_()
        self.Wv.requires_grad_()
    def forward(self,x:torch.tensor,h:int,device:str):
        self.device = torch.device(device)
        x1 = torch.reshape(x,(x.size()[0],h,x.size()[1]//h)).to(self.device)
#         print(x1.size(),self.Wq.size())
        Q = torch.einsum('ikj,kjl->ikl',x1,self.Wq).to(self.device)
        K = torch.einsum('ikj,kjl->ikl',x1,self.Wk).to(self.device)
        sqd = torch.tensor((x1.size()[2]+0.001)**0.5)
        E = torch.einsum('ijk,ljk->jil',Q,K)/sqd.to(self.device)
        A = torch.softmax(E, dim=2).to(self.device)
        V = torch.einsum('ikj,kjl->ikl',x1,self.Wv).to(self.device)
        Y = torch.einsum('ikj,jil->kil',A,V).to(self.device)
        return torch.reshape(Y,(x.size()[0],x.size()[1]))

In [17]:
class ViT(nn.Module):
    def __init__(self,p:int,h1:int,h2:int,device:str,size=3*32*32):
        super(ViT,self).__init__()
        self.ReLU = nn.ReLU
        self.device = torch.device(device)
        self.xclass = torch.randn(1,3*p*p,device=device)
        self.xclass.requires_grad_()
        self.pos = torch.randn(1+(size//(3*p*p)),3*p*p,device=device)
        self.pos.requires_grad_()
        self.atten1 = multi_attention(3*p*p,h1,device=device)
        self.norm1 = nn.LayerNorm(normalized_shape=3,device = device)
        
        
        self.atten2 = multi_attention(3*p*p,h2,device=device)
        self.norm2 = nn.LayerNorm(normalized_shape=3,device = device)
        self.mlp = nn.Linear(3*p*p,10,device=device)
        
    def forward(self,x:torch.Tensor,h1:int,h2:int,p:int,device:str):
        self.device = torch.device(device)
        x1 = torch.reshape(x,(3*p*p,(x.size()[0]*x.size()[1]*x.size()[2])//(3*p*p))).t().to(self.device)
        
        xemb = torch.cat((self.xclass,x1),dim=0)
        x2 = torch.add(xemb,self.pos)
        
        out1 = self.atten1.forward(x2,h1,device=device).to(self.device)
        out1 = torch.reshape(out1,(p*p+x.size()[1]*x.size()[2],x.size()[0]))
        out1 = torch.reshape(self.norm1(out1),x2.size())
        out1 = torch.add(x2,out1).to(self.device)
#         outl1 = self.ReLU(self.l1(out1.view(-1)))
#         out1 = torch.reshape(outl1,out1.size())
        
        out2 = self.atten2.forward(out1,h2,device=device).to(self.device)
        out2 = torch.reshape(out2,(p*p+x.size()[1]*x.size()[2],x.size()[0]))
        out2 = torch.reshape(self.norm2(out2),x2.size())
        out2 = torch.add(out2,out1).to(self.device)
#         outl2 = self.ReLU(self.l2(out2.view(-1)))
#         out2 = torch.reshape(outl2,out2.size())
        
        out3 = self.mlp(out2[0]).to(self.device)
        out3 = Fun.softmax(out3,dim=0).to(self.device)
        return out3

In [18]:
ViT_model = ViT(4,3,2,'cuda')
ViT_model.to(devic)

ViT(
  (norm1): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
  (mlp): Linear(in_features=48, out_features=10, bias=True)
)

In [19]:
epochs = 10
lr = 0.01
batch_size=32
for i in tqdm(range(epochs)):
    for j , (images, label) in enumerate(train_loader):

        images = images[0].to(devic)
        label = label.to(devic)
        
        p = ViT_model.forward(images,3,2,4,'cuda')
        lp = -torch.log(p).to(devic)

        Loss = lp[label].to(devic)
        
        if j%10000==0:
            print(Loss)
            
        Loss.backward()
        
        if((j+1)%batch_size == 0 or (j+1)==50000):
            with torch.no_grad():
                ViT_model.mlp.weight.data -= lr*(ViT_model.mlp.weight.grad)/batch_size
                ViT_model.mlp.bias.data -= lr*(ViT_model.mlp.bias.grad)/batch_size

                ViT_model.atten2.Wq.data -= lr*(ViT_model.atten2.Wq.grad)/batch_size
                ViT_model.atten2.Wk.data -= lr*(ViT_model.atten2.Wk.grad)/batch_size
                ViT_model.atten2.Wv.data -= lr*(ViT_model.atten2.Wv.grad)/batch_size

                ViT_model.atten1.Wq.data -= lr*(ViT_model.atten1.Wq.grad)/batch_size
                ViT_model.atten1.Wk.data -= lr*(ViT_model.atten1.Wk.grad)/batch_size
                ViT_model.atten1.Wv.data -= lr*(ViT_model.atten1.Wv.grad)/batch_size

                ViT_model.pos.data -= lr*(ViT_model.pos.grad)/batch_size
                ViT_model.xclass.data -= lr*(ViT_model.xclass.grad)/batch_size


            ViT_model.mlp.weight.grad.zero_()
            ViT_model.mlp.bias.grad.zero_()

            ViT_model.atten2.Wq.grad.zero_()
            ViT_model.atten2.Wk.grad.zero_()
            ViT_model.atten2.Wv.grad.zero_()

            ViT_model.atten1.Wq.grad.zero_()
            ViT_model.atten1.Wk.grad.zero_()
            ViT_model.atten1.Wv.grad.zero_()
            ViT_model.pos.grad.zero_()
            ViT_model.xclass.grad.zero_()

  0%|          | 0/10 [00:00<?, ?it/s]

tensor([2.6360], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([1.8599], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([2.2932], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([2.0129], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([2.1514], device='cuda:0', grad_fn=<IndexBackward0>)


 10%|█         | 1/10 [03:26<31:01, 206.78s/it]

tensor([1.7865], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([1.8314], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([2.3291], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([2.0232], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([2.5417], device='cuda:0', grad_fn=<IndexBackward0>)


 20%|██        | 2/10 [06:54<27:39, 207.43s/it]

...

 80%|████████  | 8/10 [27:36<06:53, 206.96s/it]

tensor([2.2786], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([1.6687], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([1.7540], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([1.9209], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([1.4763], device='cuda:0', grad_fn=<IndexBackward0>)


 90%|█████████ | 9/10 [31:08<03:28, 208.33s/it]

tensor([1.5334], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([1.1407], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([1.5215], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([3.7283], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([1.4710], device='cuda:0', grad_fn=<IndexBackward0>)


100%|██████████| 10/10 [34:36<00:00, 207.66s/it]


In [35]:

correct = torch.tensor(0)
loss = torch.tensor(0.00000001)
for i , (images, label) in enumerate(test_loader):

    images = images[0].to(devic)
    label = label.to(devic)
    if (i%1000==0) :
        print(i//1000)
    p = ViT_model.forward(images,3,2,4,'cuda')
    
    if(int(torch.argmax(p))==label):
        correct = correct+1
        
#     del out
#     torch.cuda.empty_cache()
    
test_accuracy_2 = correct/10000

0
1
2
3
4
5
6
7
8
9


In [36]:
test_accuracy_2*100

tensor(30.4400)


We can see that the accuracy of the Vision Transformer like model is lesser than that of the model involving convolutions and self attention.

It is known that encoder only models are data hungry and require a huge amount of data for them to overpower the convolutional models.